<a href="https://colab.research.google.com/github/wakii/ML_Programming/blob/master/ex5-tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- learning rate의 magic number : 2.6^4 : slice는 가장 낮은 layer와 가장 높은 layer의 learning 차이다. : discriminative learning rates
- NLP RNN에서는 2.6이 layer에서 layer로 증가할 때마다 줄어드는게 좋다.
- randomforest : NLP classifier를 이용해 예측할 때 randomforest model을 사용하였는데, 이 때 optimal parameter(LR)로 2.6을 찾았다. 



- tabular data에도 neural net을 이용하는게 효과적이라는게 밝혀졌다.
- engineering은 여전히 필요하지만 계속 simple 해졌으며, randomforest보다 neural net쓰는게 더 standard가 되었다.
- fastai.tabular는 neuralnet을 tabular data 에 쉽게 적용할 수 있게 해준다. 

# Tabular models

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32561 non-null  int64  
 1   workclass       32561 non-null  object 
 2   fnlwgt          32561 non-null  int64  
 3   education       32561 non-null  object 
 4   education-num   32074 non-null  float64
 5   marital-status  32561 non-null  object 
 6   occupation      32049 non-null  object 
 7   relationship    32561 non-null  object 
 8   race            32561 non-null  object 
 9   sex             32561 non-null  object 
 10  capital-gain    32561 non-null  int64  
 11  capital-loss    32561 non-null  int64  
 12  hours-per-week  32561 non-null  int64  
 13  native-country  32561 non-null  object 
 14  salary          32561 non-null  object 
dtypes: float64(1), int64(5), object(9)
memory usage: 3.7+ MB


In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

- data block API 이용

- `cat_names` : categorical variables
- `cont-names` : continuous variables


- 여기까지가 preprocessing

- process의 종류들
- `FillMissing` : Look for missing values and deal with them some way.
-`Categorify` : Find categorical variables and turn them into Pandas categories
-`Normalize`  : Do a normalization ahead of time which is to take continuous variables and subtract their mean and divide by their standard deviation so they are zero-one variables.

- computer vision 에서는 data들을 randomize 하기 위해 각 이미지들을 transformation 했다. : data augmentation
- tabular data에서는 transformation 대신에 processing을 하는데, 이는 transformation 과 다르게 앞에서 한 번 해준다. 


In [0]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

- `split_by_idx` : validataion set을 만들 때 index를 지정해서 만들어주어야 한다. 
  - ex) day : 연속인 날들
  - ex) video_frames : 연속인 frame처럼
  - 이 때 split by idx 가 유용하다.


- label 은 여기서 종속변수이며, 여기서는 dep_var라는 위에서 할당한 변수에서 가져옴
- 독립변수들은 cat_names, cont_names에 있다.


- 

In [14]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Federal-gov,Masters,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.1896,-0.7909,1.5334,>=50k
Private,HS-grad,Divorced,Other-service,Not-in-family,White,False,-0.4828,-0.2322,-0.4224,<50k
Private,Assoc-acdm,Married-civ-spouse,Craft-repair,Husband,White,False,-0.2629,-0.6611,0.7511,<50k
Self-emp-inc,Masters,Married-civ-spouse,Sales,Husband,White,False,1.3496,-0.6681,1.5334,>=50k
Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,False,0.4701,-0.2122,1.5334,>=50k
Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,0.7632,-0.8381,0.7511,>=50k
Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,False,-0.6294,-0.0320,1.1422,<50k
Private,HS-grad,Married-civ-spouse,Protective-serv,Husband,White,False,1.4962,0.0704,-0.4224,>=50k
?,HS-grad,Divorced,?,Not-in-family,White,False,0.5434,-1.4415,-0.4224,<50k
?,Bachelors,Married-civ-spouse,?,Husband,White,False,1.6428,-0.7222,1.1422,<50k


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [0]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.354604,0.378520,0.820000


## Inference

In [0]:
row = df.iloc[0]

In [17]:
learn.predict(row)

(Category >=50k, tensor(1), tensor([0.4698, 0.5302]))